# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
from scipy import stats
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from sklearn.metrics import mean_absolute_error
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
from itertools import chain
from pathlib import Path
import requests
from matplotlib.ticker import MaxNLocator
from regression_bias_corrector import LinearBiasCorrector
from scipy.stats import mannwhitneyu, variation, levene
from copy import deepcopy
import statsmodels.formula.api as smf


def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

def form_bar(base):
    def formatter(x):
        return f'{str(int(round(x * base)))}/{base}'
    return formatter


# V1

## Process raw data

In [ ]:
path = f"E:/YandexDisk/Work/bbd/mriya"

df_info = pd.read_excel(f"{path}/Испытуемые Яндекс.xlsx", sheet_name='Info', index_col=0)
df_info.rename(columns={'sex': 'Sex'}, inplace=True)
df_info.insert(3, 'Age', (df_info['sample_date'] - df_info['birthday']) / np.timedelta64(1, 'D') / 365.25)
df_info = df_info[df_info['Age'].notna()]

df_ifa_transform = pd.read_excel(f"{path}/data/ИФА/transformation.xlsx", index_col=0)
ifa_feats = df_ifa_transform.index.to_list()

df_ifa = pd.read_excel(f"{path}/data/ИФА/origin/ИФА_full.xlsx", sheet_name='full', skiprows=list(range(1, 8)))
new_to_old_cols = {}
for f in ifa_feats:
    new_to_old_cols[f'sample name {f}'] = f'{f} Samples'
    new_to_old_cols[f'value {f}'] = f'{f} Raw'
df_ifa.rename(columns=new_to_old_cols, inplace=True)

for f in ifa_feats[1::]:
    if not df_ifa[f"{ifa_feats[0]} Samples"].equals(df_ifa[f"{f} Samples"]):
        print(f"{ifa_feats[0]} differs from {f}")
        
for f in ifa_feats:
    df_ifa_control_points = pd.read_excel(f"{path}/data/ИФА/feats_control_points.xlsx", sheet_name=f, index_col=0)
    df_ifa_control_points['log_OD'] = np.log(df_ifa_control_points['OD, y'])
    df_ifa_control_points['log_pg_ml'] = np.log(df_ifa_control_points['pg/ml, x'])
    linreg = smf.ols(formula=f"log_pg_ml ~ log_OD", data=df_ifa_control_points).fit()
    
    df_ifa[f"log({f} Undiluted)"] = linreg.params['log_OD'] * np.log(df_ifa[f"{f} Raw"]) + linreg.params['Intercept']
    df_ifa[f"{f}"] = np.exp(df_ifa[f"log({f} Undiluted)"]) * df_ifa_transform.at[f, 'Dilution']
    df_ifa[f"log({f})"] = np.log(df_ifa[f"{f}"])
    
df_ifa.set_index(f"{ifa_feats[0]} Samples", inplace=True)
df_ifa.loc[:, list(itertools.chain.from_iterable([[f"{f} Raw", f"log({f} Undiluted)", f"{f}", f"log({f})"] for f in ifa_feats]))].to_excel(f"{path}/data/ИФА/data.xlsx", index_label='Samples')

n_rows = 2
n_cols = 5
fig_height = 6
fig_width = 18

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), sharey=False, sharex=False, gridspec_kw={'wspace':0.1, 'hspace': 0.1}, layout='constrained')

for f_id, f in enumerate(ifa_feats):
    df_ifa_control_points = pd.read_excel(f"{path}/data/ИФА/feats_control_points.xlsx", sheet_name=f, index_col=0)
    df_ifa_control_points['log_OD'] = np.log(df_ifa_control_points['OD, y'])
    df_ifa_control_points['log_pg_ml'] = np.log(df_ifa_control_points['pg/ml, x'])
    row_id, col_id = divmod(f_id, n_cols)
    df_ifa[f"log(OD)"] = np.log(df_ifa[f"{f} Raw"])
    df_ifa[f"log(pg/ml)"] = df_ifa[f"log({f} Undiluted)"]
    scatter = sns.scatterplot(
        data=df_ifa,
        x=f"log(OD)",
        y=f"log(pg/ml)",
        linewidth=0.25,
        alpha=0.75,
        edgecolor="k",
        s=20,
        color='gray',
        ax=axs[row_id, col_id],
        zorder=1,
    )
    scatter = sns.scatterplot(
        x=df_ifa_control_points["log_OD"].values,
        y=df_ifa_control_points["log_pg_ml"].values,
        linewidth=0.25,
        alpha=0.75,
        edgecolor="k",
        s=40,
        color='red',
        ax=axs[row_id, col_id],
        zorder=2,
    )
    axs[row_id, col_id].set_title(f, fontsize='large')
    # axs[n_rows - 1, n_cols - 1].axis('off') 
fig.savefig(f"{path}/data/ИФА/fit.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/data/ИФА/fit.pdf", bbox_inches='tight')
plt.close(fig)


## Check correlation with age

In [ ]:
path = f"E:/YandexDisk/Work/bbd/mriya"

df_info = pd.read_excel(f"{path}/Испытуемые Яндекс.xlsx", sheet_name='Info', index_col=0)
df_info.rename(columns={'sex': 'Sex'}, inplace=True)
df_info.insert(3, 'Age', (df_info['sample_date'] - df_info['birthday']) / np.timedelta64(1, 'D') / 365.25)
df_info = df_info[df_info['Age'].notna()]

df_ifa = pd.read_excel(f"{path}/data/ИФА/data.xlsx", index_col=0)
df_ifa.loc[df_ifa.index, 'Age'] = df_info.loc[df_ifa.index, 'Age']
df_ifa.loc[df_ifa.index, 'Риск'] = df_info.loc[df_ifa.index, 'РИСК']

df_ifa_transform = pd.read_excel(f"{path}/data/ИФА/transformation.xlsx", index_col=0)
ifa_feats = df_ifa_transform.index.to_list()

groups_ids = {
    'All': df_ifa.index.values,
    'LowRisk': df_ifa.index[df_ifa['Риск'] == 'Низкий'].values,
}

groups_feats = {
    'feats': ifa_feats,
    'log(feats)': [f"log({f})" for f in ifa_feats]
}

for g, g_ids in groups_ids.items():

    for gf in groups_feats:

        df_corr = pd.DataFrame(index=groups_feats[gf], columns=['rho'])
        for f in tqdm(groups_feats[gf]):
            df_tmp = df_ifa.loc[g_ids, ['Age', f]].dropna(axis=0, how='any')
            if df_tmp.shape[0] > 1:
                vals_1 = df_tmp.loc[:, 'Age'].values
                vals_2 = df_tmp.loc[:, f].values
                df_corr.at[f, 'rho'], _ = stats.pearsonr(vals_1, vals_2)
        df_corr.dropna(axis=0, how='any', inplace=True)
        df_corr.insert(1, "abs(rho)", df_corr['rho'].abs())
        df_corr.sort_values(["abs(rho)"], ascending=[False], inplace=True)
        feats_cnt_wo_age = df_corr.index.to_list()
        feats_cnt = ['Age'] + feats_cnt_wo_age
        df_corr = df_corr.apply(pd.to_numeric)

        sns.set_theme(style='ticks')
        fig, ax = plt.subplots(figsize=(0.5 + 0.3 * df_corr.index.str.len().max(), 0.5 + 0.4 * len(feats_cnt_wo_age) + 0.04 * df_corr.index.str.len().max()) , layout='constrained')
        heatmap = sns.heatmap(
            df_corr.loc[:, ['rho']],
            annot=True,
            fmt=".2f",
            vmin=-1.0,
            vmax=1.0,
            cmap='coolwarm',
            linewidth=0.1,
            linecolor='black',
            #annot_kws={"fontsize": 15},
            cbar_kws={
                # "shrink": 0.9,
                # "aspect": 30,
                #'fraction': 0.046, 
                #'pad': 0.04,
            },
            ax=ax
        )
        heatmap_pos = ax.get_position()
        ax.figure.axes[-1].set_position([heatmap_pos.x1 + 0.05, heatmap_pos.y0, 0.1, heatmap_pos.height])
        ax.figure.axes[-1].set_ylabel(r"Pearson $\rho$")
        for spine in ax.figure.axes[-1].spines.values():
            spine.set(visible=True, lw=0.25, edgecolor="black")
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set(xticklabels=[])
        ax.set(xticks=[])
        plt.savefig(f"{path}/data/ИФА/age_{g}_{gf}_pearsonr.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path}/data/ИФА/age_{g}_{gf}_pearsonr.pdf", bbox_inches='tight')
        plt.close(fig)

## Low vs High risk

In [ ]:
pheno_associations = {
    'Риск': {
        'groups': ['Низкий', 'Высокий'],
        'base': 'Низкий',
        'colors': {'Низкий': 'dodgerblue', 'Высокий': 'crimson'}
    },
}


df_imms = df_ifa[['Age', 'Риск'] + ifa_feats + [f"log({f})" for f in ifa_feats]].copy()

feats_sets = {
    'lin': ifa_feats,
    'log': [f"log({f})" for f in ifa_feats]
}

for fs in feats_sets:
    
    feats_curr = feats_sets[fs]

    for an_col in pheno_associations:
        pathlib.Path(f"{path}/data/ИФА/{an_col}").mkdir(parents=True, exist_ok=True)
        
        df_imms_ass = df_imms.loc[df_imms[an_col].isin(pheno_associations[an_col]['groups'])]
        df_imms_stat = pd.DataFrame(index=feats_curr)
        an_col_str = an_col.replace(' ', '_')
        an_col_str = an_col_str.replace('-', '_')
        df_imms_ass[an_col_str] = df_imms_ass[an_col]
        an_vals = pheno_associations[an_col]['groups']
        an_val_base = pheno_associations[an_col]['base']
        an_colors = pheno_associations[an_col]['colors']
        
        for f_id, f in enumerate(feats_curr):
            f_str = f.replace(' ', '_')
            f_str = f_str.replace('(', '')
            f_str = f_str.replace(')', '')
            df_imms_ass[f_str] = df_imms_ass[f]
            vals = {}
            for group in an_vals:
                vals[group] = df_imms_ass.loc[df_imms_ass[an_col] == group, f].values
                df_imms_stat.at[f, f"Mean {group}"] = np.mean(vals[group])
                df_imms_stat.at[f, f"Median {group}"] = np.median(vals[group])
                df_imms_stat.at[f, f"Q75 {group}"], df_imms_stat.at[f, f"Q25 {group}"] = np.percentile(vals[group], [75 , 25])
                df_imms_stat.at[f, f"IQR {group}"] = df_imms_stat.at[f, f"Q75 {group}"] - df_imms_stat.at[f, f"Q25 {group}"]
                df_imms_stat.at[f, f"Variation {group}"] = variation(vals[group])
            _, df_imms_stat.at[f, "mannwhitneyu_pval"] = mannwhitneyu(vals[an_vals[0]], vals[an_vals[1]], alternative='two-sided')
            _, df_imms_stat.at[f, "levene_pval"] = levene(vals[an_vals[0]], vals[an_vals[1]])
            regcov = smf.ols(formula=f"{f_str} ~ {an_col_str} + Age", data=df_imms_ass).fit()
            reg_sum = regcov.summary2().tables[1]
            pvals_cols = reg_sum.index[reg_sum.index.str.contains(an_col_str)].values
            for pval_col_id, pval_col in enumerate(pvals_cols):
                df_imms_stat.at[f, f"ancova_{pval_col}_pval"] = reg_sum.at[pval_col, 'P>|t|']
        _, df_imms_stat.loc[feats_curr, "mannwhitneyu_pval_fdr_bh"], _, _ = multipletests(df_imms_stat.loc[feats_curr, "mannwhitneyu_pval"].values, 0.05, method='fdr_bh')
        _, df_imms_stat.loc[feats_curr, "mannwhitneyu_pval_bonferroni"], _, _ = multipletests(df_imms_stat.loc[feats_curr, "mannwhitneyu_pval"].values, 0.05, method='bonferroni')
        _, df_imms_stat.loc[feats_curr, "mannwhitneyu_pval_simes-hochberg"], _, _ = multipletests(df_imms_stat.loc[feats_curr, "mannwhitneyu_pval"].values, 0.05, method='simes-hochberg')
        _, df_imms_stat.loc[feats_curr, "levene_pval_fdr_bh"], _, _ = multipletests(df_imms_stat.loc[feats_curr, "levene_pval"].values, 0.05, method='fdr_bh')
        _, df_imms_stat.loc[feats_curr, "levene_pval_bonferroni"], _, _ = multipletests(df_imms_stat.loc[feats_curr, "levene_pval"].values, 0.05, method='bonferroni')
        _, df_imms_stat.loc[feats_curr, "levene_pval_simes-hochberg"], _, _ = multipletests(df_imms_stat.loc[feats_curr, "levene_pval"].values, 0.05, method='simes-hochberg')
        pvals_cols_ancova = df_imms_stat.columns[df_imms_stat.columns.str.contains(an_col_str)].values
        for pval_col in pvals_cols_ancova:
            _, df_imms_stat.loc[feats_curr, f"{pval_col}_fdr_bh"], _, _ = multipletests(df_imms_stat.loc[feats_curr, pval_col].values, 0.05, method='fdr_bh')
            _, df_imms_stat.loc[feats_curr, f"{pval_col}_bonferroni"], _, _ = multipletests(df_imms_stat.loc[feats_curr, pval_col].values, 0.05, method='bonferroni')
            _, df_imms_stat.loc[feats_curr, f"{pval_col}_simes-hochberg"], _, _ = multipletests(df_imms_stat.loc[feats_curr, pval_col].values, 0.05, method='simes-hochberg')
        df_imms_stat.sort_values([f"mannwhitneyu_pval"], ascending=[True], inplace=True)
        df_imms_stat.to_excel(f"{path}/data/ИФА/{an_col}/imms_{fs}.xlsx")
        
        for stat_test in [x.replace('_pval', '') for x in pvals_cols_ancova] + ['mannwhitneyu', 'levene']:
            df_fig = df_imms_stat.copy()
            df_fig.sort_values([f"{stat_test}_pval"], ascending=[True], inplace=True)
            df_fig['Features'] = df_fig.index
            df_fig[f'{stat_test}_pval_fdr_bh_log'] = -np.log10(df_fig[f'{stat_test}_pval_fdr_bh'])
            df_fig['color'] = 'pink'
            df_fig.loc[df_fig[f'{stat_test}_pval_fdr_bh'] < 0.05, 'color'] = 'red'
            sns.set_theme(style='ticks')
            fig, ax = plt.subplots(figsize=(3, df_fig.shape[0] * 0.5))
            barplot = sns.barplot(
                data=df_fig,
                y='Features',
                x=f'{stat_test}_pval_fdr_bh_log',
                edgecolor='black',
                palette=df_fig['color'].values,
                ax=ax,
            )
            ax.set_xlabel(r"$-\log_{10}(\mathrm{p-value})$")
            ax.xaxis.tick_top()
            ax.xaxis.set_label_position('top')
            ax.set_ylabel('')
            plt.savefig(f"{path}/data/ИФА/{an_col}/imms_pvals_{stat_test}_{fs}.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path}/data/ИФА/{an_col}/imms_pvals_{stat_test}_{fs}.pdf", bbox_inches='tight')
            plt.close(fig)
        
        n_rows = 2
        n_cols = 5
        fig_width = 19
        fig_height = 9
        
        sns.set_theme(style='ticks')
        fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={'wspace':0.15, 'hspace': 0.15}, layout='constrained')
        for f_id, f in enumerate(df_imms_stat.index.values):
            row_id, col_id = divmod(f_id, n_cols)
            
            ql = df_imms_ass[f].quantile(0.01)
            qh = df_imms_ass[f].quantile(0.99)
            
            sns.violinplot(
                data=df_imms_ass,
                x=an_col,
                y=f,
                palette=an_colors,
                scale='width',
                order=an_vals,
                saturation=0.75,
                ax=axs[row_id, col_id],
                legend=False,
                cut=0,
            )
            axs[row_id, col_id].set_ylabel(f)
            axs[row_id, col_id].ticklabel_format(style='scientific', scilimits=(-1, 1), axis='y', useOffset=True, useMathText=True)
            mannwhitneyu_pval = df_imms_stat.at[f, "mannwhitneyu_pval_fdr_bh"]
            levene_pval = df_imms_stat.at[f, "levene_pval_fdr_bh"]
            title = f'Mann-Whitney: {mannwhitneyu_pval:.2e}\nLevene: {levene_pval:.2e}'
            for pval_col in pvals_cols_ancova:
                title += f"\nANCOVA: {df_imms_stat.at[f, pval_col + '_fdr_bh']:.2e}"
            axs[row_id, col_id].set_title(title)

        fig.savefig(f"{path}/data/ИФА/{an_col}/imms_distribution_{fs}.png", bbox_inches='tight', dpi=200)
        fig.savefig(f"{path}/data/ИФА/{an_col}/imms_distribution_{fs}.pdf", bbox_inches='tight')
        plt.close(fig)

## Old IFA for our data

In [ ]:
path = f"E:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/data"

df_imm = pd.read_excel(f"{path}/immuno/data.xlsx", index_col=0)
df_ifa = pd.read_excel(f"{path}/ИФА/Results FGF23 FGF21 Klotho GDF15.xlsx", index_col=0)
df_ifa.loc[df_ifa.index, ['Age', 'Status']] = df_imm.loc[df_ifa.index, ['Age', 'Status']]

feats = ['FGF23', 'FGF21', 'KLOTHO', 'GDF15']

df_corr = pd.DataFrame(index=feats, columns=['rho'])
for f in tqdm(feats):
    df_tmp = df_ifa.loc[df_ifa['Status'] == 'Control', ['Age', f]].dropna(axis=0, how='any')
    if df_tmp.shape[0] > 1:
        vals_1 = df_tmp.loc[:, 'Age'].values
        vals_2 = df_tmp.loc[:, f].values
        df_corr.at[f, 'rho'], _ = stats.pearsonr(vals_1, vals_2)
df_corr.dropna(axis=0, how='any', inplace=True)
df_corr.insert(1, "abs(rho)", df_corr['rho'].abs())
df_corr.sort_values(["abs(rho)"], ascending=[False], inplace=True)
df_corr = df_corr.apply(pd.to_numeric)

sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(0.5 + 0.3 * df_corr.index.str.len().max(), 0.5 + 0.4 * len(feats) + 0.04 * df_corr.index.str.len().max()) , layout='constrained')
heatmap = sns.heatmap(
    df_corr.loc[:, ['rho']],
    annot=True,
    fmt=".2f",
    vmin=-1.0,
    vmax=1.0,
    cmap='coolwarm',
    linewidth=0.1,
    linecolor='black',
    #annot_kws={"fontsize": 15},
    cbar_kws={
        # "shrink": 0.9,
        # "aspect": 30,
        #'fraction': 0.046, 
        #'pad': 0.04,
    },
    ax=ax
)
heatmap_pos = ax.get_position()
ax.figure.axes[-1].set_position([heatmap_pos.x1 + 0.05, heatmap_pos.y0, 0.1, heatmap_pos.height])
ax.figure.axes[-1].set_ylabel(r"Pearson $\rho$")
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
ax.set_xlabel('')
ax.set_ylabel('')
ax.set(xticklabels=[])
ax.set(xticks=[])
plt.savefig(f"{path}/ИФА/age_{gf}_pearsonr.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/ИФА/age_{gf}_pearsonr.pdf", bbox_inches='tight')
plt.close(fig)

# v0

## Process raw data

In [ ]:
path = f"E:/YandexDisk/Work/bbd/mriya"

df_info = pd.read_excel(f"{path}/Испытуемые Яндекс.xlsx", sheet_name='Info', index_col=0)
df_info.rename(columns={'sex': 'Sex'}, inplace=True)
df_info.insert(3, 'Age', (df_info['sample_date'] - df_info['birthday']) / np.timedelta64(1, 'D') / 365.25)
df_info = df_info[df_info['Age'].notna()]

df_ifa_transform = pd.read_excel(f"{path}/data/ИФА/transformation.xlsx", index_col=0)
ifa_feats = df_ifa_transform.index.to_list()

df_ifa_raw = pd.read_excel(f"{path}/data/ИФА/raw.xlsx", skiprows=list(range(1, 14)))

for f in ifa_feats[1::]:
    if not df_ifa_raw[f"{ifa_feats[0]} Samples"].equals(df_ifa_raw[f"{f} Samples"]):
        print(f"{ifa_feats[0]} differs from {f}")

for f in ifa_feats:
    df_ifa_raw[f"log({f} Undiluted)"] = df_ifa_transform.at[f, 'Slope'] * np.log(df_ifa_raw[f"{f} Raw"]) + df_ifa_transform.at[f, 'Intercept']
    df_ifa_raw[f"{f}"] = np.exp(df_ifa_raw[f"log({f} Undiluted)"]) * df_ifa_transform.at[f, 'Dilution']
    df_ifa_raw[f"log({f})"] = np.log(df_ifa_raw[f"{f}"])
    
df_ifa_raw.set_index(f"{ifa_feats[0]} Samples", inplace=True)
df_ifa_raw.loc[:, list(itertools.chain.from_iterable([[f"{f} Raw", f"log({f} Undiluted)", f"{f}", f"log({f})"] for f in ifa_feats]))].to_excel(f"{path}/data/ИФА/data.xlsx", index_label='Samples')

n_rows = 2
n_cols = 4
fig_height = 10
fig_width = 18

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={}, sharey=False, sharex=False, layout='constrained')

for f_id, f in enumerate(ifa_feats):
    df_ifa_control_points = pd.read_excel(f"{path}/data/ИФА/feats_control_points.xlsx", sheet_name=f, index_col=0)
    row_id, col_id = divmod(f_id, n_cols)
    df_ifa_raw[f"log(x)"] = np.log(df_ifa_raw[f"{f} Raw"])
    df_ifa_raw[f"log(y)"] = df_ifa_raw[f"log({f} Undiluted)"]
    scatter = sns.scatterplot(
        data=df_ifa_raw,
        x=f"log(x)",
        y=f"log(y)",
        linewidth=0.25,
        alpha=0.75,
        edgecolor="k",
        s=20,
        color='gray',
        ax=axs[row_id, col_id],
        zorder=1,
    )
    scatter = sns.scatterplot(
        x=df_ifa_control_points["log(x)"].values,
        y=df_ifa_control_points["log(y)"].values,
        linewidth=0.25,
        alpha=0.75,
        edgecolor="k",
        s=40,
        color='red',
        ax=axs[row_id, col_id],
        zorder=2,
    )
    axs[row_id, col_id].set_title(f, fontsize='large')
    # axs[n_rows - 1, n_cols - 1].axis('off') 
fig.savefig(f"{path}/data/ИФА/fit.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/data/ИФА/fit.pdf", bbox_inches='tight')
plt.close(fig)

## Check correlation with age

In [ ]:
path = f"E:/YandexDisk/Work/bbd/mriya"

df_info = pd.read_excel(f"{path}/Испытуемые Яндекс.xlsx", sheet_name='Info', index_col=0)
df_info.rename(columns={'sex': 'Sex'}, inplace=True)
df_info.insert(3, 'Age', (df_info['sample_date'] - df_info['birthday']) / np.timedelta64(1, 'D') / 365.25)
df_info = df_info[df_info['Age'].notna()]

df_ifa = pd.read_excel(f"{path}/data/ИФА/data.xlsx", index_col=0)
df_ifa.loc[df_ifa.index, 'Age'] = df_info.loc[df_ifa.index, 'Age']

df_ifa_transform = pd.read_excel(f"{path}/data/ИФА/transformation.xlsx", index_col=0)
ifa_feats = df_ifa_transform.index.to_list()

groups_feats = {
    'feats': ifa_feats,
    'log(feats)': [f"log({f})" for f in ifa_feats]
}

for gf in groups_feats:

    df_corr = pd.DataFrame(index=groups_feats[gf], columns=['rho'])
    for f in tqdm(groups_feats[gf]):
        df_tmp = df_ifa.loc[:, ['Age', f]].dropna(axis=0, how='any')
        if df_tmp.shape[0] > 1:
            vals_1 = df_tmp.loc[:, 'Age'].values
            vals_2 = df_tmp.loc[:, f].values
            df_corr.at[f, 'rho'], _ = stats.pearsonr(vals_1, vals_2)
    df_corr.dropna(axis=0, how='any', inplace=True)
    df_corr.insert(1, "abs(rho)", df_corr['rho'].abs())
    df_corr.sort_values(["abs(rho)"], ascending=[False], inplace=True)
    feats_cnt_wo_age = df_corr.index.to_list()
    feats_cnt = ['Age'] + feats_cnt_wo_age
    df_corr = df_corr.apply(pd.to_numeric)

    sns.set_theme(style='ticks')
    fig, ax = plt.subplots(figsize=(0.5 + 0.3 * df_corr.index.str.len().max(), 0.5 + 0.4 * len(feats_cnt_wo_age) + 0.04 * df_corr.index.str.len().max()) , layout='constrained')
    heatmap = sns.heatmap(
        df_corr.loc[:, ['rho']],
        annot=True,
        fmt=".2f",
        vmin=-1.0,
        vmax=1.0,
        cmap='coolwarm',
        linewidth=0.1,
        linecolor='black',
        #annot_kws={"fontsize": 15},
        cbar_kws={
            # "shrink": 0.9,
            # "aspect": 30,
            #'fraction': 0.046, 
            #'pad': 0.04,
        },
        ax=ax
    )
    heatmap_pos = ax.get_position()
    ax.figure.axes[-1].set_position([heatmap_pos.x1 + 0.05, heatmap_pos.y0, 0.1, heatmap_pos.height])
    ax.figure.axes[-1].set_ylabel(r"Pearson $\rho$")
    for spine in ax.figure.axes[-1].spines.values():
        spine.set(visible=True, lw=0.25, edgecolor="black")
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set(xticklabels=[])
    ax.set(xticks=[])
    plt.savefig(f"{path}/data/ИФА/age_{gf}_pearsonr.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/data/ИФА/age_{gf}_pearsonr.pdf", bbox_inches='tight')
    plt.close(fig)